# Preparación de base google trends


Para armar la solicitud de términos de búsqueda se utiliza la librería `pytrends`, cuya forma de utilización se puede encontrar en [***hackernoon***](https://hackernoon.com/how-to-use-google-trends-api-with-python).

In [ ]:
!pip install pytrends 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=40ac8c3fa5863828052ccaded8d01c37d234890589504e822010d5958923f8b6
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [ ]:
from functools import partial, reduce # to transform variable dic in a dataframe  
import pandas as pd

In [ ]:
def daily_gt(keyword, start, end, inputCategories ,inputCategoriesNames , hl='en-US', tz=360):
  import time
  import pandas as pd 
  import numpy as np # numpy y pandas to data wrangling 
  from datetime import datetime, timedelta # to work w date
  from pytrends.request import TrendReq
  # función para dividir rango de fechas en segmentos 
  def date_range(start, end, intv):
      start = datetime.strptime(start,"%Y-%m-%d")
      end = datetime.strptime(end,"%Y-%m-%d")
      diff = (end  - start ) / intv
      for i in range(intv):
          yield (start + diff * i).strftime("%Y-%m-%d")
      yield end.strftime("%Y-%m-%d")

  # set pytrends para region y zona horaria
  pytrends = TrendReq(hl=hl, tz=tz) 

  # generación de lista de fechas a utilizar
  firstDate = datetime.strptime(start,"%Y-%m-%d")
  lastDate = datetime.strptime(end,"%Y-%m-%d")
  diffDays_control = lastDate - firstDate 
  if  diffDays_control.days >= 90:
    aux = (lastDate - firstDate)/90
    intv = aux.days
    timelist = list(date_range(start, end, intv))
  else:
    timelist = list([start, end])

  # armamos lista vacía para guardar los resultados
  var_dict={}

  # loop de palabras o categorías a importar
  for x in range(0, len(keyword)):
    varName = keyword[x]
    print(f'{x}: {varName}')
    dataset = pd.DataFrame(columns = [varName])
    # loop de rango de fechas sobre cada palabra o categoría
    for i in range(0, len(timelist)-1):
      if timelist[i] != start:
        startAux = datetime.strptime(timelist[i], "%Y-%m-%d") + timedelta(days=1)
        startNew = startAux.strftime("%Y-%m-%d")
      else:
        startAux = datetime.strptime(timelist[i], "%Y-%m-%d")
        startNew = startAux.strftime("%Y-%m-%d")
      print(f'Iteration from {startNew} to {timelist[i+1]}\n')

      if type(varName)==int: # para considerar las categorías por separado
        pytrends.build_payload(kw_list=[''], cat=varName, timeframe=f'{startNew} {timelist[i+1]}') 
        data = pytrends.interest_over_time()
        loc = inputCategories.index(varName)
        catName = inputCategoriesNames[loc] # para renombrar columnas sin nombres en categorías
        data.rename(columns = {f'':f'{catName}'}, inplace = True)

      else:
        pytrends.build_payload(kw_list=[varName], cat=0, timeframe=f'{startNew} {timelist[i+1]}') 
        data = pytrends.interest_over_time()

      if not data.empty: # chequear que la base importada no esté vacía antes de trabajarla, sino pasar a la prox palabra
        data = pd.DataFrame(data.drop(labels=['isPartial'],axis='columns'))
        data['year'] = data.index.year
        data['month'] = data.index.month 
        data['day'] = data.index.day 
        dataset = dataset.append(data)
        del data
        time.sleep(2) # para aumentar el tiempo entre iteración para evitar el eror 429
      else: 
        continue      
    if type(varName)==int:
      dataset = dataset.iloc[:,1:] # elinamos la columna vacía que se genera por importar categorías que no coinciden con el nombre de la primer columna del dataset generado al ppio
      pytrends.build_payload(kw_list=[''], cat=varName, timeframe='all') 
      historical_data = pytrends.interest_over_time()
      historical_data.rename(columns = {f'':f'{catName}_historical'}, inplace = True)
      varName = catName # para que encuentre las columnas para hacer la normalización
    else:
      pytrends.build_payload(kw_list=[varName], cat=0, timeframe='all') 
      historical_data = pytrends.interest_over_time()
      historical_data.rename(columns = {f'{varName}':f'{varName}_historical'}, inplace = True)

    historical_data = pd.DataFrame(historical_data.drop(labels=['isPartial'], axis='columns'))
    historical_data['year'] = historical_data.index.year
    historical_data['month'] = historical_data.index.month 
      
    dataset = pd.merge(dataset, historical_data, on=["year", "month"])
    del historical_data
    dataset[f'{varName}_adjusted'] = dataset[f'{varName}']*(dataset[f'{varName}_historical']/100) 
    var_dict[varName] = pd.DataFrame(dataset)
  return var_dict

In [ ]:
crypto = ['cryptocurrency', 'crypto', 'bitcoin', 'bitcoin price', 'ethereum', 'ethereum price']
monetary = ['stock market', 'wall street', 'interest rate', 'fed', 'bankruptcy']
real = ['taxes', 'investment']
policy = ['china', 'united states', 'war', 'russia']
influencers = ['elon musk', 'do kwon']
topics = ['/m/0vpj4_b', '/m/05p0rrx', '/m/0g_fl', '/m/0108bn2x','/m/0965sb', 
          '/g/1214g6vy', '/m/0drqp', '/m/09jx2', '/m/07g82', '/m/0f10yl','/g/11j2cc_qll','/m/061s4']
topicsNames = ['cryptocurrency_top', 'bitcoin_top', 'investment_top', 
               'ethereum_top','exchange_top', 'bankrup_top', 
               'stock_market_top', 'inflation', 'taxes', 'digital_wallet_top', 'covid19', 'pandemic']
categories = [904, 37, 814]
categoriesNames = ['future_commodities', 'banking', 'foreign_currency'] 
kw_list = crypto + monetary + policy + topics + influencers + categories
kw_list

['cryptocurrency',
 'crypto',
 'bitcoin',
 'bitcoin price',
 'ethereum',
 'ethereum price',
 'stock market',
 'wall street',
 'interest rate',
 'fed',
 'bankruptcy',
 'china',
 'united states',
 'war',
 'russia',
 '/m/0vpj4_b',
 '/m/05p0rrx',
 '/m/0g_fl',
 '/m/0108bn2x',
 '/m/0965sb',
 '/g/1214g6vy',
 '/m/0drqp',
 '/m/09jx2',
 '/m/07g82',
 '/m/0f10yl',
 '/g/11j2cc_qll',
 '/m/061s4',
 'elon musk',
 'do kwon',
 904,
 37,
 814]

In [ ]:
start = '2015-08-01' # parece ser el límite inferior de precios de bitcoin en yahoo finance
end = '2022-06-15'
var_dict = daily_gt(keyword = kw_list, start = start, end = end, inputCategories=categories, inputCategoriesNames=categoriesNames)

0: cryptocurrency
Iteration from 2015-08-01 to 2015-11-01

Iteration from 2015-11-02 to 2016-02-02

Iteration from 2016-02-03 to 2016-05-05

Iteration from 2016-05-06 to 2016-08-06

Iteration from 2016-08-07 to 2016-11-07

Iteration from 2016-11-08 to 2017-02-08

Iteration from 2017-02-09 to 2017-05-12

Iteration from 2017-05-13 to 2017-08-13

Iteration from 2017-08-14 to 2017-11-14

Iteration from 2017-11-15 to 2018-02-15

Iteration from 2018-02-16 to 2018-05-19

Iteration from 2018-05-20 to 2018-08-20

Iteration from 2018-08-21 to 2018-11-21

Iteration from 2018-11-22 to 2019-02-22

Iteration from 2019-02-23 to 2019-05-26

Iteration from 2019-05-27 to 2019-08-27

Iteration from 2019-08-28 to 2019-11-28

Iteration from 2019-11-29 to 2020-02-29

Iteration from 2020-03-01 to 2020-06-01

Iteration from 2020-06-02 to 2020-09-02

Iteration from 2020-09-03 to 2020-12-04

Iteration from 2020-12-05 to 2021-03-07

Iteration from 2021-03-08 to 2021-06-08

Iteration from 2021-06-09 to 2021-09-09

In [ ]:
var_dict.values()

dict_values([     cryptocurrency    year  month   day  cryptocurrency_historical  \
0                46  2015.0    8.0   1.0                          0   
1                69  2015.0    8.0   2.0                          0   
2                52  2015.0    8.0   3.0                          0   
3                81  2015.0    8.0   4.0                          0   
4                68  2015.0    8.0   5.0                          0   
...             ...     ...    ...   ...                        ...   
2505             25  2022.0    6.0  10.0                         20   
2506             25  2022.0    6.0  11.0                         20   
2507             36  2022.0    6.0  12.0                         20   
2508             73  2022.0    6.0  13.0                         20   
2509              0  2022.0    6.0  14.0                         20   

     cryptocurrency_adjusted  
0                        0.0  
1                        0.0  
2                        0.0  
3         

In [ ]:
from functools import partial, reduce # to transform variable dic in a dataframe  
# merge todos los dataframes en el diccionario var_dict
my_reduce = partial(pd.merge, on=['year','month','day'], how='outer') 
gtrends = reduce(my_reduce, var_dict.values())
# modificamos las columnas con los tópicos según sus respectivos nombres 
for i in range(0, len(topics)): 
  dtcol = [col for col in gtrends.columns if topics[i] in col]
  for ex in dtcol:
    num = len(topics[i])+1
    word = ex[num:None]
    gtrends.rename(columns={ex: f'{topicsNames[i]}_' + word}, inplace=True)
# generamos la variable fecha para tener como índice
gtrends['Date'] = pd.to_datetime(gtrends[["year", "month", "day"]])
gtrends.set_index('Date', inplace=True)
del var_dict
gtrends

,cryptocurrency,year,month,day,cryptocurrency_historical,cryptocurrency_adjusted,crypto,crypto_historical,crypto_adjusted,bitcoin,...,do kwon_adjusted,future_commodities,future_commodities_historical,future_commodities_adjusted,banking,banking_historical,banking_adjusted,foreign_currency,foreign_currency_historical,foreign_currency_adjusted
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-01,46,2015.0,8.0,1.0,0,0.0,63,1,0.63,100,...,7.0,60.0,45,27.00,69.0,92,63.48,50.0,52,26.00
2015-08-02,69,2015.0,8.0,2.0,0,0.0,64,1,0.64,77,...,6.3,51.0,45,22.95,54.0,92,49.68,47.0,52,24.44
2015-08-03,52,2015.0,8.0,3.0,0,0.0,72,1,0.72,73,...,5.9,72.0,45,32.40,100.0,92,92.00,67.0,52,34.84
2015-08-04,81,2015.0,8.0,4.0,0,0.0,90,1,0.9,72,...,8.5,71.0,45,31.95,95.0,92,87.40,67.0,52,34.84
2015-08-05,68,2015.0,8.0,5.0,0,0.0,79,1,0.79,64,...,8.6,71.0,45,31.95,93.0,92,85.56,66.0,52,34.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,25,2022.0,6.0,10.0,20,5.0,19,34,6.46,29,...,0.9,65.0,77,50.05,94.0,76,71.44,95.0,90,85.50
2022-06-11,25,2022.0,6.0,11.0,20,5.0,21,34,7.14,31,...,0.9,50.0,77,38.50,66.0,76,50.16,73.0,90,65.70
2022-06-12,36,2022.0,6.0,12.0,20,7.2,24,34,8.16,38,...,0.9,46.0,77,35.42,55.0,76,41.80,66.0,90,59.40


In [ ]:
from google.colab import drive
drive.mount('drive')
gtrends.to_excel('gtrends.xlsx')
!cp gtrends.xlsx "/content/drive/MyDrive/ds/proyecto/dataset/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
